## Settings

In [ ]:
# insert your desired path to work on
import os
from os.path import join
project_path = os.path.dirname(os.getcwd())
os.chdir(join('..','data'))
os.getcwd()

Run the following section one time per session. This cell links the code folder to the python exectution path.

In [2]:
import sys
sys.path.append(join(project_path, 'code'))

The following cell allows Jupyter Notebooks to detect changes in external code and to automatically update it without restarting the runtime.

In [3]:
%load_ext autoreload
%autoreload 2

Plots settings.

In [6]:
import matplotlib
font = {'family':'Arial', 'size':'15', 'weight':'normal'}

matplotlib.rc('font', **font)

Set folder structure.

In [ ]:
config = {
    'main_brazil': 'Brazil',
    'main_peru': 'Peru',
    'baseline': join(project_path, "baseline_models"),
    'output': join(project_path, "code", "saved_models"),
    'metrics': join(project_path, "code", "metrics")
}

# List comprehension for the folder structure code
[os.makedirs(val, exist_ok=True) for key, val in config.items()]

# **AI4Dengue forecasting**
![](https://drive.google.com/uc?export=view&id=1J5Bt5Cks-e2IV-dEJLHJkuwXFJNFAZgr)

In [8]:
import utils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime
from glob import glob
from configPeru import DEP_NAMES, GROUPED_VARS, DATA_REDUCER_SETTINGS, DATA_PROCESSING_SETTINGS

# Data

## Load the dataframe
**This dataframe comprises all the variables (climatic, epidemiological etc.) acquired for each Department during a defined number of years.**

In [ ]:
dataframe = pd.read_csv(join(config['main_Peru'], "Peru_Departments_dengue_monthly.csv"))
dataframe.head()

**'Clean' the dataset (e.g. remove NaN values)**

In [ ]:
dataframe.rename(columns={'PopTotal':'PopTotal_UF'}, inplace=True)
dataframe.rename(columns={'Pop0_19':'Pop0_19_UF'}, inplace=True)
dataframe.rename(columns={'PopTotal_Urban':'PopTotal_Urban_UF'}, inplace=True)
dataframe.rename(columns={'PopTotal_Rural':'PopTotal_Rural_UF'}, inplace=True)
dataframe['Pop0_19_Urban_UF'] = dataframe['PopTotal_Urban_UF']
dataframe['Pop0_19_Rural_UF'] = dataframe['PopTotal_Rural_UF']

deps = pd.unique(dataframe['ADM1_PCODE'])
deps_dictionary = { i : dep for dep,i in enumerate(deps) }
dataframe['ADM1_PCODE'] = dataframe['ADM1_PCODE'].map(deps_dictionary) 

dataframe = utils.clean(dataframe)
dataframe.head()

In [ ]:
dataframe.info()

## Apply Data Reduction
**Data reduction is applied to three macro groups in order to reduce the number of variables on which the AI framework will be trained. The variables belonging to each group are set with the *PCAgroups* dictionary. The groups are:**
1. ***CLIMATIC VARIABLES***,
2. ***GEO VARIABLES***,
3. ***SOCIO VARIABLES***

In [ ]:
print('\033[1m PCA Excluded Variables \033[0m')
utils.plist(GROUPED_VARS['EXCLUDED'])

print('\033[1m Climatic variables \033[0m')
utils.plist(GROUPED_VARS['CLIMATIC VARIABLES'])

print('\033[1m Geo variables \033[0m')
utils.plist(GROUPED_VARS['GEO VARIABLES'])

print('\033[1m Socio variables \033[0m')
utils.plist(GROUPED_VARS['SOCIO VARIABLES'])

print('\033[1m Additional variables \033[0m')
utils.plist(GROUPED_VARS['AUXILIAR'])

print('\033[1m Dengue variables \033[0m')
utils.plist(GROUPED_VARS['DENGUE'])

**We selected two types of data reduction methods: PCA (Principal Component Analysis) and PLS (Principal Least Square). The second one is the default solution because it reduces the input data by considering also a second variable that in our case is the Dengue Incidence Rates.**

In [14]:
from data_reduction import pca_reducer, pls_reducer

**Extract climatic, geophysical and socio-economic variables from the dataframe**

In [15]:
X_climatic = dataframe[GROUPED_VARS['CLIMATIC VARIABLES']].values
X_geo = dataframe[GROUPED_VARS['GEO VARIABLES']].values
X_socio = dataframe[GROUPED_VARS['SOCIO VARIABLES']].values

**Extract Dengue variables from the dataframe, apply a root scaling and normalization**

In [16]:
y_dengue = dataframe[GROUPED_VARS['DENGUE']].values
scaler = MinMaxScaler()
y_dengue = scaler.fit_transform(y_dengue)
#y_dengue = np.power(y_dengue, 1/4)

**Apply data reduction technique**

In [ ]:
if DATA_REDUCER_SETTINGS['TYPE'] == 'PLS':    
    climatic_vars_reduced = pls_reducer(
        X_climatic,
        y_dengue,
        DATA_REDUCER_SETTINGS['NUMBER OF COMPONENTS']['CLIMATIC VARIABLES'])
    
    geo_vars_reduced = pls_reducer(
        X_geo,
        y_dengue,
        DATA_REDUCER_SETTINGS['NUMBER OF COMPONENTS']['GEO VARIABLES'])
    
    socio_vars_reduced = pls_reducer(
        X_socio,
        y_dengue,
        DATA_REDUCER_SETTINGS['NUMBER OF COMPONENTS']['SOCIO VARIABLES'])
    
elif DATA_REDUCER_SETTINGS['TYPE'] == 'PCA':
    climatic_vars_reduced = pca_reducer(
        X_climatic,
        DATA_REDUCER_SETTINGS['NUMBER OF COMPONENTS']['CLIMATIC VARIABLES'])
    
    geo_vars_reduced = pca_reducer(
        X_geo,
        DATA_REDUCER_SETTINGS['NUMBER OF COMPONENTS']['GEO VARIABLES'])
    
    socio_vars_reduced = pca_reducer(
        X_socio,
        DATA_REDUCER_SETTINGS['NUMBER OF COMPONENTS']['SOCIO VARIABLES'])
else:
    print('No data reduction.')

## Order reduced data in a new dataframe


**Normalize remaining variables**

In [18]:
x_excluded = dataframe[GROUPED_VARS['EXCLUDED']].values
x_excluded = MinMaxScaler().fit_transform(x_excluded)

In [19]:
X_auxiliar = dataframe[GROUPED_VARS['AUXILIAR']].values
X_auxiliar = MinMaxScaler().fit_transform(X_auxiliar)

**Create a new database with the reduced, the auxiliar and Dengue variables**

In [ ]:
independent = {'Year':dataframe.Year.values, 'dep_id':dataframe.ADM1_PCODE.values, 't_fundc_ocup18m':np.zeros((len(dataframe.ADM1_PCODE.values))), 't_medioc_ocup18m':np.zeros((len(dataframe.ADM1_PCODE.values))),
               'PopTotal_Urban_UF':x_excluded[:, 0], 'PopTotal_Rural_UF':x_excluded[:, 1], 'total_precipitation_d':x_excluded[:, 2], 
               'surface_pressure_d':x_excluded[:, 3], 'area_km2':x_excluded[:, 4], 'humidity_d':x_excluded[:, 5], 'temperature_2m_d':x_excluded[:, 6],
               'min_temperature_2m_d':x_excluded[:, 7]}

auxiliar    = {'Month': X_auxiliar[:, 0], 
               'cases20_99': X_auxiliar[:, 1], 'cases0_19': X_auxiliar[:, 2],
               'RandEffects1':  MinMaxScaler().fit_transform(np.reshape(dataframe.ADM1_PCODE.values*dataframe.Month.values, (dataframe.ADM1_PCODE.values.shape[0], 1)))[:,0], 
               'RandEffects2':  MinMaxScaler().fit_transform(np.reshape(dataframe.ADM1_PCODE.values*dataframe.Year.values, (dataframe.ADM1_PCODE.values.shape[0], 1)))[:,0], 
               'RandEffects3':  MinMaxScaler().fit_transform(np.reshape(dataframe.ADM1_PCODE.values*dataframe.Month.values*dataframe.Year.values, (dataframe.ADM1_PCODE.values.shape[0], 1)))[:,0]}

climatic    = {'PCA0-Climatic':climatic_vars_reduced[:,0], 'PCA1-Climatic':climatic_vars_reduced[:,1], 'PCA2-Climatic':climatic_vars_reduced[:,2], 
               'PCA3-Climatic':climatic_vars_reduced[:,3]}

geo         = {'PCA0-Geo':geo_vars_reduced[:,0], 'PCA1-Geo':geo_vars_reduced[:,1], 'PCA2-Geo':geo_vars_reduced[:,2], 
               'PCA3-Geo':geo_vars_reduced[:,3], 'PCA4-Geo':geo_vars_reduced[:,4], 'PCA5-Geo':geo_vars_reduced[:,5]}

socio       = {'PCA0-Socio':socio_vars_reduced[:,0], 'PCA1-Socio':socio_vars_reduced[:,1], 'PCA2-Socio':socio_vars_reduced[:,2], 
               'PCA3-Socio':socio_vars_reduced[:,3], 'PCA4-Socio':socio_vars_reduced[:,4], 'PCA5-Socio':socio_vars_reduced[:,5]}

dengue      = {'DengRate_all': y_dengue[:,0], 'DengRate_019': y_dengue[:,1]}

columns     = {**independent, **auxiliar, **climatic, **geo, **socio, **dengue}

reduced_dataframe = pd.DataFrame(columns)
reduced_dataframe.head()

## Create training and validation data
**First of all, the dataframe is divided in two sub-dataframes (training and validation) by using the variable *Year***

In [21]:
training_dataframe   = reduced_dataframe[reduced_dataframe.Year <= 2016]
validation_dataframe = reduced_dataframe[reduced_dataframe.Year >= 2016]

**Then the dataset handler is initialized. This object will handle all the operations needed to create, reshape and augment the training and validation dataset to fit the requirements of each Deep Learning or Machine Learning model.**

In [22]:
from datasetHandler import datasetHandler
dataset_handler = datasetHandler(training_dataframe, validation_dataframe)

**Get training and validation vectors from dataframes.**

In [ ]:
x_train, y_train, x_val, y_val = dataset_handler.get_data(DATA_PROCESSING_SETTINGS['T LEARNING'], DATA_PROCESSING_SETTINGS['T PREDICTION'])
print('\n\nX Training shape', x_train.shape)
print('Y Training shape', y_train.shape)
print('X Validation shape', x_val.shape)
print('Y Validation shape', y_val.shape)

**Apply data augmention**

In [ ]:
x_train_a, y_train_a, x_val_a, y_val_a = dataset_handler.augment(x_train, y_train, x_val, y_val, DATA_PROCESSING_SETTINGS['AUGMENTATION'])
print('X Training shape', x_train_a.shape)
print('Y Training shape', y_train_a.shape)
print('X Validation shape', x_val_a.shape)
print('Y Validation shape', y_val_a.shape)

# Modelling

## Baseline
**Upload results from Baseline Model**

In [25]:
deps_dictionary

{'PE16': 0, 'PE17': 1, 'PE20': 2, 'PE22': 3, 'PE24': 4, 'PE25': 5}

In [ ]:
baselineAll = pd.read_csv(join(config['baseline'], "Peru", "predicted_All_Cases.csv")).drop('Unnamed: 0', axis=1)
baselineAll['Date'] = pd.to_datetime(baselineAll['Date'])

cols = ['pred.uci','pred.mean','pred.lci']
baselineAll.loc[:, cols] = baselineAll.loc[:, cols].div(baselineAll['PopTotal'], axis=0).multiply(100000.0, axis=0)
baselineAll['state_index'] = baselineAll['ADM1_PCODE'].map(deps_dictionary)

baselineAll.head()

In [ ]:
baseline019 = pd.read_csv(join(config['baseline'], "Peru", "predicted_0_19.csv")).drop('Unnamed: 0', axis=1)
baseline019['Date'] = pd.to_datetime(baseline019['Date'])

cols = ['pred.uci','pred.mean','pred.lci']
baseline019.loc[:, cols] = baseline019.loc[:, cols].div(baseline019['Pop0_19'], axis=0).multiply(100000.0, axis=0)
baseline019['state_index'] = baseline019['ADM1_PCODE'].map(deps_dictionary)
baseline019.head(10)

## CatBoost
**Prepare dataset to fit CatBoost requirements**

In [28]:
trainingC, validationC = dataset_handler.prepare_data_CatBoost(x_train_a[:,:,2:], y_train_a, x_val_a[:,:,2:], y_val_a)

**Initilize the CatBoost**

In [29]:
from models import CatBoostNet
cat_boost = CatBoostNet()

**Train or Test**

In [30]:
def compute_scores(gt, preds):
    # Compute Normalized-RMSE and R2 metrics
    rmse = mean_squared_error(gt, preds, squared=False)
    nrmse = rmse/(gt.max() - gt.min())
    r2 = r2_score(gt, preds)
    return nrmse, r2

In [ ]:
TRAINING = False
nb_deps = 6

if TRAINING:
    # get model trained on Brazil
    cat_boost_model = glob(join(config['output'], "Brazil", "catboost*"))
    if cat_boost_model == []:
        print('No file with such pattern was found in the directory.')
    else:
        cat_boost.load(cat_boost_model[0])
        
        # finetune pre-trained model in Peru
        cat_boost.train(trainingC, validationC, output_path=join(config['output'], "Peru"))

else:    
    # retrieve finetuned model
    cat_boost_model_finetuned = glob(join(config['output'], "Peru", "catboost*"))
    if cat_boost_model_finetuned == []:
        print('No file with such pattern was found in the directory. Run TRAINING = True first.')
    else:
        cat_boost.load(cat_boost_model_finetuned[0])

        trainC, valC = dataset_handler.prepare_data_CatBoost(x_train[:,:,2:], y_train, x_val[:,:,2:], y_val)

        preds_tra = cat_boost.model.predict(trainC[0])
        preds_tra[preds_tra < 0] = 0
        preds_val = cat_boost.model.predict(valC[0])
        preds_val[preds_val < 0] = 0

        datelist = pd.date_range('01-01-2010', end='01-01-2020', freq='YS')
        dates = datelist.strftime("%Y").tolist()

        # Total timesteps (nb of months inside this time window) = 228
        month_datelist = pd.date_range('01-01-2010', end='31-12-2019', freq='MS')
        ts = np.arange(0, len(month_datelist), 1)

        res = []
        res.append('Department,NRMSE 0-19 Training,NRMSE All Training,NRMSE 0-19 Validation,NRMSE All Validation')

        for i in range(nb_deps):
            fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(18,9))

            idxt = preds_tra.shape[0]//nb_deps
            idxv = preds_val.shape[0]//nb_deps

            t1 = ts[12:12+idxt]
            t2 = ts[(12+idxt):(12+idxt+idxv)]

            # Reverting Data Norm ---------
            pt = scaler.inverse_transform(preds_tra[idxt*i:idxt*(i+1), :])
            pv = scaler.inverse_transform(preds_val[idxv*i:idxv*(i+1), :])

            gtt = scaler.inverse_transform(trainC[1][idxt*i:idxt*(i+1), :])
            gtv = scaler.inverse_transform(valC[1][idxv*i:idxv*(i+1), :])
            # ------------------------------

            baseAll_df = baselineAll[baselineAll.state_index == i][['pred.mean','pred.uci','pred.lci']]
            base019_df = baseline019[baseline019.state_index == i][['pred.mean','pred.uci','pred.lci']]

            axes[0].plot(t1, gtt[:,0], '-', color='orange', label = 'Observed Cases')
            axes[0].plot(t2, gtv[:,0], '-', color='orange')
            axes[0].plot(t1, pt[:,0], '--', color='red', label = 'CatBoost prediction on Training data')
            axes[0].plot(t2, pv[:,0], '-', color='red', label = 'CatBoost prediction on Validation data')

            # Baseline model
            axes[0].plot(t1, baseAll_df['pred.mean'].iloc[t1], color='forestgreen', label = 'Baseline model prediction')
            axes[0].fill_between(x=t1, y1=baseAll_df['pred.lci'].iloc[t1], y2=baseAll_df['pred.uci'].iloc[t1], facecolor='forestgreen', alpha=.1, label='Baseline model Confidence Interval')
            axes[0].plot(t2, baseAll_df['pred.mean'].iloc[t2], color='forestgreen')
            axes[0].fill_between(x=t2, y1=baseAll_df['pred.lci'].iloc[t2], y2=baseAll_df['pred.uci'].iloc[t2], facecolor='forestgreen', alpha=.1)

            axes[1].plot(t1, gtt[:,1], '-', color='orange', label = 'Observed Cases')
            axes[1].plot(t2, gtv[:,1], '-', color='orange')
            axes[1].plot(t1, pt[:,1], '--', color='red', label = 'CatBoost prediction on Training data')
            axes[1].plot(t2, pv[:,1], '-', color='red', label = 'CatBoost prediction on Validation data')

            # Baseline model
            axes[1].plot(t1, base019_df['pred.mean'].iloc[t1], color='forestgreen', label = 'Baseline model prediction')
            axes[1].fill_between(x=t1, y1=base019_df['pred.lci'].iloc[t1], y2=base019_df['pred.uci'].iloc[t1], facecolor='forestgreen', alpha=.1, label='Baseline model Confidence Interval')
            axes[1].plot(t2, base019_df['pred.mean'].iloc[t2], color='forestgreen')
            axes[1].fill_between(x=t2, y1=base019_df['pred.lci'].iloc[t2], y2=base019_df['pred.uci'].iloc[t2], facecolor='forestgreen', alpha=.1)

            # Configure visualization
            axes[0].set_title(DEP_NAMES[i], fontsize = 18, loc='left')
            axes[0].set(ylabel='DIR Total Population')
            axes[1].set(xlabel='Year', ylabel='DIR 0-19')
            axes[1].legend(ncol=3, loc='lower center', bbox_to_anchor=(0.5,-0.5), shadow=True)
            axes[0].axvline(t2[0], color='black', linewidth=2, linestyle='--', alpha=0.5)
            axes[1].axvline(t2[0], color='black', linewidth=2, linestyle='--', alpha=0.5)

            for ax in axes:
                #ax.set_xticks(np.arange(0, idxt+idxv, 11))
                ax.set_xticks(np.arange(0, len(ts)+1, 12))
                ax.grid(True, linewidth=0.5)
                ax.set_xlim(ts[0],ts[-1]+5)
            axes[0].set_xticklabels([])
            axes[1].set_xticklabels(dates)

            fig.tight_layout()
            plt.show()
            plt.close()

            print('--------------------- TRAINING scores ---------------------')
            rmse1, r2 = compute_scores(gtt[:,0], pt[:,0])
            print('N-RMSE (group total): ', rmse1)
            print('R2   (group total): ', r2)

            rmse2, r2 = compute_scores(gtt[:,1], pt[:,1])
            print('N-RMSE (group 0-19): ', rmse2)
            print('R2   (group 0-19): ', r2)

            print('-------------------- VALIDATION scores ---------------------')
            rmse3, r2 = compute_scores(gtv[:,0], pv[:,0])
            print('N-RMSE (group total): ', rmse3)
            print('R2   (group total): ', r2)

            rmse4, r2 = compute_scores(gtv[:,1], pv[:,1])
            print('N-RMSE (group 0-19): ', rmse4)
            print('R2   (group 0-19): ', r2)

            res.append('{},{},{},{},{}'.format(DEP_NAMES[i], rmse2, rmse1, rmse4, rmse3))

            print('\n##########################################################################################################')

        # Save results
        today = datetime.now().strftime("%d-%m-%Y-%H:%M:%S")
        with open(join(config['metrics'], "Peru", 'catboost-'+today+'.csv'), 'w') as f:
            print('Writing performance metrics to file...')
            for item in res:
                f.write("%s\n" % item)

Write file with performance metrics for baseline model.

In [ ]:
cat_boost_model_finetuned = glob(join(config['output'], "Peru", "catboost*"))[0]
cat_boost.load(cat_boost_model_finetuned)

trainC, valC = dataset_handler.prepare_data_CatBoost(x_train[:,:,2:], y_train, x_val[:,:,2:], y_val)
preds_tra = cat_boost.model.predict(trainC[0])
preds_tra[preds_tra < 0] = 0
preds_val = cat_boost.model.predict(valC[0])
preds_val[preds_val < 0] = 0

datelist = pd.date_range('01-01-2010', end='01-01-2020', freq='YS')
dates = datelist.strftime("%Y").tolist()
# Total timesteps (nb of months inside this time window) = 228
month_datelist = pd.date_range('01-01-2010', end='31-12-2019', freq='MS')
ts = np.arange(0, len(month_datelist), 1)

res = []
res.append('Department,NRMSE 0-19 Training,NRMSE All Training,NRMSE 0-19 Validation,NRMSE All Validation')

for i in range(nb_deps):
    idxt = preds_tra.shape[0]//nb_deps
    idxv = preds_val.shape[0]//nb_deps 

    t1 = ts[12:12+idxt]
    t2 = ts[(12+idxt):(12+idxt+idxv)]

    # Reverting Data Norm ---------
    gtt = scaler.inverse_transform(trainC[1][idxt*i:idxt*(i+1), :])
    gtv = scaler.inverse_transform(valC[1][idxv*i:idxv*(i+1), :])
    # ------------------------------
    
    baseAll_df = baselineAll[baselineAll.state_index == i][['pred.mean','pred.uci','pred.lci']]
    base019_df = baseline019[baseline019.state_index == i][['pred.mean','pred.uci','pred.lci']]

    print('--------------------- TRAINING scores ---------------------')
    rmse1, r2 = compute_scores(gtt[:,0], baseAll_df['pred.mean'].iloc[t1])
    print('N-RMSE (group total): ', rmse1)
    print('R2   (group total): ', r2)

    rmse2, r2 = compute_scores(gtt[:,1], base019_df['pred.mean'].iloc[t1])
    print('N-RMSE (group 0-19): ', rmse2)
    print('R2   (group 0-19): ', r2)

    print('-------------------- VALIDATION scores ---------------------')
    rmse3, r2 = compute_scores(gtv[:,0], baseAll_df['pred.mean'].iloc[t2])
    print('N-RMSE (group total): ', rmse3)
    print('R2   (group total): ', r2)

    rmse4, r2 = compute_scores(gtv[:,1], base019_df['pred.mean'].iloc[t2])
    print('N-RMSE (group 0-19): ', rmse4)
    print('R2   (group 0-19): ', r2)

    res.append('{},{},{},{},{}'.format(DEP_NAMES[i], rmse2, rmse1, rmse4, rmse3))
    
    print('\n##########################################################################################################')

# Save results
today = datetime.now().strftime("%d-%m-%Y-%H:%M:%S")
with open(join(config['metrics'], "Peru", 'baseline-'+today+'.csv'),'w') as f:
    print('Writing performance metrics to file...')
    for item in res:
        f.write("%s\n" % item)

## SVM
**It uses the same data structure of CatBoost, no need to prepare data**

**Initilize the SVM**

In [32]:
from models import SVMNet
svm = SVMNet()

**Train or Test**

In [ ]:
TRAINING = False

if TRAINING:
    
    # get model trained on Brazil
    svm_model = glob(join(config['output'], "Brazil", "svm*"))
    if svm_model == []:
        print('No file with such pattern was found in the directory.')
    else:
        svm.load(svm_model[0])
        
        # finetune pre-trained model in Peru
        svm.train(trainingC, validationC, output_path=join(config['output'], "Peru"))

else:
    # retrieve finetuned model
    svm_model_finetuned = glob(join(config['output'], "Peru", "svm*"))
    if svm_model_finetuned == []:
        print('No file with such pattern was found in the directory. Run TRAINING = True first.')
    else:
        svm.load(svm_model_finetuned[0])

        trainC, valC = dataset_handler.prepare_data_CatBoost(x_train[:,:,2:], y_train, x_val[:,:,2:], y_val)

        preds_tra = svm.model.predict(trainC[0])
        preds_tra[preds_tra < 0] = 0
        preds_val = svm.model.predict(valC[0])
        preds_val[preds_val < 0] = 0

        datelist = pd.date_range('01-01-2010', end='01-01-2020', freq='YS')
        dates = datelist.strftime("%Y").tolist()
        month_datelist = pd.date_range('01-01-2010', end='31-12-2019', freq='MS')
        ts = np.arange(0, len(month_datelist), 1)

        res = []
        res.append('Department,NRMSE 0-19 Training,NRMSE All Training,NRMSE 0-19 Validation,NRMSE All Validation')

        for i in range(nb_deps):
            fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(18,9))

            idxt = preds_tra.shape[0]//nb_deps
            idxv = preds_val.shape[0]//nb_deps

            t1 = ts[12:12+idxt]
            t2 = ts[(12+idxt):(12+idxt+idxv)]

            # Reverting Data Norm ---------
            pt = scaler.inverse_transform(preds_tra[idxt*i:idxt*(i+1), :])
            pv = scaler.inverse_transform(preds_val[idxv*i:idxv*(i+1), :])

            gtt = scaler.inverse_transform(trainC[1][idxt*i:idxt*(i+1), :])
            gtv = scaler.inverse_transform(valC[1][idxv*i:idxv*(i+1), :])
            # ------------------------------

            baseAll_df = baselineAll[baselineAll.state_index == i][['pred.mean','pred.uci','pred.lci']]
            base019_df = baseline019[baseline019.state_index == i][['pred.mean','pred.uci','pred.lci']]

            axes[0].plot(t1, gtt[:,0], '-', color='orange', label = 'Observed Cases')
            axes[0].plot(t2, gtv[:,0], '-', color='orange')
            axes[0].plot(t1, pt[:,0], '--', color='red', label = 'SVM prediction on Training data')
            axes[0].plot(t2, pv[:,0], '-', color='red', label = 'SVM prediction on Validation data')

            # Baseline model
            axes[0].plot(t1, baseAll_df['pred.mean'].iloc[t1], color='forestgreen', label = 'Baseline model prediction')
            axes[0].fill_between(x=t1, y1=baseAll_df['pred.lci'].iloc[t1], y2=baseAll_df['pred.uci'].iloc[t1], facecolor='forestgreen', alpha=.1, label='Baseline model Confidence Interval')
            axes[0].plot(t2, baseAll_df['pred.mean'].iloc[t2], color='forestgreen')
            axes[0].fill_between(x=t2, y1=baseAll_df['pred.lci'].iloc[t2], y2=baseAll_df['pred.uci'].iloc[t2], facecolor='forestgreen', alpha=.1)

            axes[1].plot(t1, gtt[:,1], '-', color='orange', label = 'Observed Cases')
            axes[1].plot(t2, gtv[:,1], '-', color='orange')
            axes[1].plot(t1, pt[:,1], '--', color='red', label = 'SVM prediction on Training data')
            axes[1].plot(t2, pv[:,1], '-', color='red', label = 'SVM prediction on Validation data')

            # Baseline model
            axes[1].plot(t1, base019_df['pred.mean'].iloc[t1], color='forestgreen', label = 'Baseline model prediction')
            axes[1].fill_between(x=t1, y1=base019_df['pred.lci'].iloc[t1], y2=base019_df['pred.uci'].iloc[t1], facecolor='forestgreen', alpha=.1, label='Baseline model Confidence Interval')
            axes[1].plot(t2, base019_df['pred.mean'].iloc[t2], color='forestgreen')
            axes[1].fill_between(x=t2, y1=base019_df['pred.lci'].iloc[t2], y2=base019_df['pred.uci'].iloc[t2], facecolor='forestgreen', alpha=.1)

            # Configure visualization
            axes[0].set_title(DEP_NAMES[i], fontsize = 18, loc='left')
            axes[0].set(ylabel='DIR Total Population')
            axes[1].set(xlabel='Year', ylabel='DIR 0-19')
            axes[1].legend(ncol=3, loc='lower center', bbox_to_anchor=(0.5,-0.5), shadow=True)
            axes[0].axvline(t2[0], color='black', linewidth=2, linestyle='--', alpha=0.5)
            axes[1].axvline(t2[0], color='black', linewidth=2, linestyle='--', alpha=0.5)

            for ax in axes:
                ax.set_xticks(np.arange(0, len(ts)+1, 12))
                ax.grid(True, linewidth=0.5)
                ax.set_xlim(ts[0],ts[-1]+5)
            axes[0].set_xticklabels([])
            axes[1].set_xticklabels(dates)

            fig.tight_layout()
            plt.show()
            plt.close()

            print('--------------------- TRAINING scores ---------------------')
            rmse1, r2 = compute_scores(gtt[:,0], pt[:,0])
            print('N-RMSE (group total): ', rmse1)
            print('R2   (group total): ', r2)

            rmse2, r2 = compute_scores(gtt[:,1], pt[:,1])
            print('N-RMSE (group 0-19): ', rmse2)
            print('R2   (group 0-19): ', r2)

            print('-------------------- VALIDATION scores ---------------------')
            rmse3, r2 = compute_scores(gtv[:,0], pv[:,0])
            print('N-RMSE (group total): ', rmse3)
            print('R2   (group total): ', r2)

            rmse4, r2 = compute_scores(gtv[:,1], pv[:,1])
            print('N-RMSE (group 0-19): ', rmse4)
            print('R2   (group 0-19): ', r2)

            res.append('{},{},{},{},{}'.format(DEP_NAMES[i], rmse2, rmse1, rmse4, rmse3))

            print('\n##########################################################################################################')

        # Save results
        today = datetime.now().strftime("%d-%m-%Y-%H:%M:%S")
        with open(join(config['metrics'], "Peru", 'svm-'+today+'.csv'), 'w') as f:
            print('Writing performance metrics to file...')
            for item in res:
                f.write("%s\n" % item)

## LSTM
**Prepare dataset to fit LSTM requirements**

In [33]:
trainingL, validationL = dataset_handler.prepare_data_LSTM(x_train_a[:,:,2:], y_train_a, x_val_a[:,:,2:], y_val_a)

**Initilize the LSTM**

In [34]:
from models import LSTMNet
lstm = LSTMNet(trainingL[0].shape[1:])

**Train or Test**

In [ ]:
TRAINING = False

if TRAINING:
    
    # get model trained on Brazil
    lstm_model = glob(join(config['output'], "Brazil", "lstm*"))
    if lstm_model == []:
        print('No file with such pattern was found in the directory.')
    else:
        lstm.load(lstm_model[0])
        
        # finetune pre-trained model in Peru
        lstm.train(trainingL, validationL, output_path=join(config['output'], "Peru"))

else:
    # retrieve finetuned model
    lstm_model_finetuned = glob(join(config['output'], "Peru", "lstm*"))
    if lstm_model_finetuned == []:
        print('No file with such pattern was found in the directory. Run TRAINING = True first.')
    else:
        lstm.load(lstm_model_finetuned[0])

        trainL, valL = dataset_handler.prepare_data_LSTM(x_train[:,:,2:], y_train, x_val[:,:,2:], y_val)

        preds_tra = lstm.model.predict(trainL[0])
        preds_tra[preds_tra < 0] = 0
        preds_val = lstm.model.predict(valL[0])
        preds_val[preds_val < 0] = 0

        #datelist = pd.date_range('01-01-2001', end='31-12-2019', freq = 'Y')
        datelist = pd.date_range('01-01-2010', end='01-01-2020', freq='YS')
        dates = datelist.strftime("%Y").tolist()
        month_datelist = pd.date_range('01-01-2010', end='31-12-2019', freq='MS')
        ts = np.arange(0, len(month_datelist), 1)

        res = []
        res.append('Department,NRMSE 0-19 Training,NRMSE All Training,NRMSE 0-19 Validation,NRMSE All Validation')

        for i in range(nb_deps):
            fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(18,9))

            idxt = preds_tra.shape[0]//nb_deps
            idxv = preds_val.shape[0]//nb_deps

            t1 = ts[12:12+idxt]
            t2 = ts[(12+idxt):(12+idxt+idxv)]

            # Reverting Data Norm ---------
            pt = scaler.inverse_transform(preds_tra[idxt*i:idxt*(i+1), :])
            pv = scaler.inverse_transform(preds_val[idxv*i:idxv*(i+1), :])

            gtt = scaler.inverse_transform(trainL[1][idxt*i:idxt*(i+1), :])
            gtv = scaler.inverse_transform(valL[1][idxv*i:idxv*(i+1), :])
            # ------------------------------

            baseAll_df = baselineAll[baselineAll.state_index == i][['pred.mean','pred.uci','pred.lci']]
            base019_df = baseline019[baseline019.state_index == i][['pred.mean','pred.uci','pred.lci']]

            axes[0].plot(t1, gtt[:,0], '-', color='orange', label = 'Observed Cases')
            axes[0].plot(t2, gtv[:,0], '-', color='orange')
            axes[0].plot(t1, pt[:,0], '--', color='red', label = 'LSTM prediction on Training data')
            axes[0].plot(t2, pv[:,0], '-', color='red', label = 'LSTM prediction on Validation data')

            # Baseline model
            axes[0].plot(t1, baseAll_df['pred.mean'].iloc[t1], color='forestgreen', label = 'Baseline model prediction')
            axes[0].fill_between(x=t1, y1=baseAll_df['pred.lci'].iloc[t1], y2=baseAll_df['pred.uci'].iloc[t1], facecolor='forestgreen', alpha=.1, label='Baseline model Confidence Interval')
            axes[0].plot(t2, baseAll_df['pred.mean'].iloc[t2], color='forestgreen')
            axes[0].fill_between(x=t2, y1=baseAll_df['pred.lci'].iloc[t2], y2=baseAll_df['pred.uci'].iloc[t2], facecolor='forestgreen', alpha=.1)

            axes[1].plot(t1, gtt[:,1], '-', color='orange', label = 'Observed Cases')
            axes[1].plot(t2, gtv[:,1], '-', color='orange')
            axes[1].plot(t1, pt[:,1], '--', color='red', label = 'LSTM prediction on Training data')
            axes[1].plot(t2, pv[:,1], '-', color='red', label = 'LSTM prediction on Validation data')

            # Baseline model
            axes[1].plot(t1, base019_df['pred.mean'].iloc[t1], color='forestgreen', label = 'Baseline model prediction')
            axes[1].fill_between(x=t1, y1=base019_df['pred.lci'].iloc[t1], y2=base019_df['pred.uci'].iloc[t1], facecolor='forestgreen', alpha=.1, label='Baseline model Confidence Interval')
            axes[1].plot(t2, base019_df['pred.mean'].iloc[t2], color='forestgreen')
            axes[1].fill_between(x=t2, y1=base019_df['pred.lci'].iloc[t2], y2=base019_df['pred.uci'].iloc[t2], facecolor='forestgreen', alpha=.1)

            # Configure visualization
            axes[0].set_title(DEP_NAMES[i], fontsize = 18, loc='left')
            axes[0].set(ylabel='DIR Total Population')
            axes[1].set(xlabel='Year', ylabel='DIR 0-19')
            axes[1].legend(ncol=3, loc='lower center', bbox_to_anchor=(0.5,-0.5), shadow=True)
            axes[0].axvline(t2[0], color='black', linewidth=2, linestyle='--', alpha=0.5)
            axes[1].axvline(t2[0], color='black', linewidth=2, linestyle='--', alpha=0.5)

            for ax in axes:
                ax.set_xticks(np.arange(0, len(ts)+1, 12))
                ax.grid(True, linewidth=0.5)
                ax.set_xlim(ts[0],ts[-1]+5)
            axes[0].set_xticklabels([])
            axes[1].set_xticklabels(dates)

            fig.tight_layout()
            plt.show()
            plt.close()

            print('--------------------- TRAINING scores ---------------------')
            rmse1, r2 = compute_scores(gtt[:,0], pt[:,0])
            print('N-RMSE (group total): ', rmse1)
            print('R2   (group total): ', r2)

            rmse2, r2 = compute_scores(gtt[:,1], pt[:,1])
            print('N-RMSE (group 0-19): ', rmse2)
            print('R2   (group 0-19): ', r2)

            print('-------------------- VALIDATION scores ---------------------')
            rmse3, r2 = compute_scores(gtv[:,0], pv[:,0])
            print('N-RMSE (group total): ', rmse3)
            print('R2   (group total): ', r2)

            rmse4, r2 = compute_scores(gtv[:,1], pv[:,1])
            print('N-RMSE (group 0-19): ', rmse4)
            print('R2   (group 0-19): ', r2)

            res.append('{},{},{},{},{}'.format(DEP_NAMES[i], rmse2, rmse1, rmse4, rmse3))

            print('\n##########################################################################################################')

        # Save results
        today = datetime.now().strftime("%d-%m-%Y-%H:%M:%S")
        with open(join(config['metrics'], "Peru", 'lstm-'+today+'.csv'), 'w') as f:
            print('Writing performance metrics to file...')
            for item in res:
                f.write("%s\n" % item)

## Ensemble
**Prepare dataset to fit the Ensemble requirements**

In [35]:
from scipy.optimize import curve_fit
from tqdm import tqdm

In [36]:
def interpolation(trainL, valL):

    def pol_zero(x, a):
        return a 

    def pol_one(x, a, b):
        return a + b*x

    def pol_two(x, a, b, c):
        return a + b*x + c*x*x  

    train_019_pol0 = []
    train_019_pol1 = []
    train_019_pol2 = []

    train_all_pol0 = []
    train_all_pol1 = []
    train_all_pol2 = []

    val_019_pol0 = []
    val_019_pol1 = []
    val_019_pol2 = []

    val_all_pol0 = []
    val_all_pol1 = []
    val_all_pol2 = []

    for i in tqdm(range(trainL[0].shape[0])):
        popt_0, _ = curve_fit(pol_zero, np.arange(0,12,1), trainL[0][i,:,-1])
        popt_1, _ = curve_fit(pol_one, np.arange(0,12,1),  trainL[0][i,:,-1])
        popt_2, _ = curve_fit(pol_two, np.arange(0,12,1), trainL[0][i,:,-1])

        train_019_pol0.append(pol_zero(12, *popt_0))
        train_019_pol1.append(pol_one(12, *popt_1))
        train_019_pol2.append(pol_two(12, *popt_2))

        popt_0, _ = curve_fit(pol_zero, np.arange(0,12,1), trainL[0][i,:,-2])
        popt_1, _ = curve_fit(pol_one, np.arange(0,12,1),  trainL[0][i,:,-2])
        popt_2, _ = curve_fit(pol_two, np.arange(0,12,1),  trainL[0][i,:,-2])

        train_all_pol0.append(pol_zero(12, *popt_0))
        train_all_pol1.append(pol_one(12, *popt_1))
        train_all_pol2.append(pol_two(12, *popt_2))

    for i in tqdm(range(valL[0].shape[0])):
        popt_0, _ = curve_fit(pol_zero, np.arange(0,12,1), valL[0][i,:,-1])
        popt_1, _ = curve_fit(pol_one, np.arange(0,12,1), valL[0][i,:,-1])
        popt_2, _ = curve_fit(pol_two, np.arange(0,12,1), valL[0][i,:,-1])

        val_019_pol0.append(pol_zero(12, *popt_0))
        val_019_pol1.append(pol_one(12, *popt_1))
        val_019_pol2.append(pol_two(12, *popt_2))

        popt_0, _ = curve_fit(pol_zero, np.arange(0,12,1), valL[0][i,:,-2])
        popt_1, _ = curve_fit(pol_one, np.arange(0,12,1), valL[0][i,:,-2])
        popt_2, _ = curve_fit(pol_two, np.arange(0,12,1), valL[0][i,:,-2])

        val_all_pol0.append(pol_zero(12, *popt_0))
        val_all_pol1.append(pol_one(12, *popt_1))
        val_all_pol2.append(pol_two(12, *popt_2))

    train_019_pol0 = np.array(train_019_pol0)
    train_019_pol1 = np.array(train_019_pol1)
    train_019_pol2 = np.array(train_019_pol2)

    train_all_pol0 = np.array(train_all_pol0)
    train_all_pol1 = np.array(train_all_pol1)
    train_all_pol2 = np.array(train_all_pol2)

    val_019_pol0 = np.array(val_019_pol0)
    val_019_pol1 = np.array(val_019_pol1)
    val_019_pol2 = np.array(val_019_pol2)

    val_all_pol0 = np.array(val_all_pol0)
    val_all_pol1 = np.array(val_all_pol1)
    val_all_pol2 = np.array(val_all_pol2)


    train_pol0 = np.column_stack((train_019_pol0, train_all_pol0))
    train_pol1 = np.column_stack((train_019_pol1, train_all_pol1))
    train_pol2 = np.column_stack((train_019_pol2, train_all_pol2))

    val_pol0 = np.column_stack((val_019_pol0, val_all_pol0))
    val_pol1 = np.column_stack((val_019_pol1, val_all_pol1))
    val_pol2 = np.column_stack((val_019_pol2, val_all_pol2))

    return train_pol0, train_pol1, train_pol2, val_pol0, val_pol1, val_pol2

**Initilize the Ensemble model**

In [37]:
from models import Ensamble, CatBoostEnsableNet, RandomForestEnsableNet
ens = RandomForestEnsableNet(True) #CatBoostEnsableNet() #Ensemble(2)

**Train or Test**

In [ ]:
# Load Brazilian inner models
cat_boost.load(glob(join(config['output'], "Brazil", "catboost*"))[0])
svm.load(glob(join(config['output'], "Brazil", "svm*"))[0])
lstm.load(glob(join(config['output'], "Brazil", "lstm*"))[0])

# Load finetuned inner models
# cat_boost.load(glob(join(config['output'], "Peru", "catboost*"))[0])
# svm.load(glob(join(config['output'], "Peru", "svm*"))[0])
# lstm.load(glob(join(config['output'], "Peru", "lstm*"))[0])

In [ ]:
TRAINING = False

if TRAINING:
    #ens.train([catBoost_train, svm_train, lstm_train], trainingL[1], 
    #          [catBoost_val, svm_val, lstm_val], validationL[1], output_path=join(config['output'], "Peru"))
    
    # get model trained on Brazil
    ens_model = glob(join(config['output'], "Brazil", "rf*"))
    if ens_model == []:
        print('No file with such pattern was found in the directory.')
    else:
        ens.load(ens_model[0])
        
        catBoost_train = cat_boost.model.predict(trainingC[0])
        catBoost_train[catBoost_train < 0] = 0
        catBoost_val = cat_boost.model.predict(validationC[0])
        catBoost_val[catBoost_val < 0] = 0

        svm_train = svm.model.predict(trainingC[0])
        svm_train[svm_train < 0] = 0
        svm_val = svm.model.predict(validationC[0])
        svm_val[svm_val < 0] = 0

        lstm_train = lstm.model.predict(trainingL[0])
        lstm_train[lstm_train < 0] = 0
        lstm_val = lstm.model.predict(validationL[0])
        lstm_val[lstm_val < 0] = 0

        train_pol0, train_pol1, train_pol2, val_pol0, val_pol1, val_pol2 = interpolation(trainingL, validationL)

        x_trainE = np.concatenate((catBoost_train, svm_train, lstm_train, train_pol0, train_pol1, train_pol2), axis=-1)
        y_trainE = trainingL[1]
        x_valE = np.concatenate((catBoost_val, svm_val, lstm_val, val_pol0, val_pol1, val_pol2), axis=-1)
        y_valE = validationL[1]

        ens.train(x_trainE, y_trainE, x_valE, y_valE, output_path=join(config['output'], "Peru"))

else:
    # retrieve finetuned RF model
    ens_model_finetuned = glob(join(config['output'], "Peru", "rf*"))
    if ens_model_finetuned == []:
        print('No file with such pattern was found in the directory. Run TRAINING = True first.')
    else:
        ens.load(ens_model_finetuned[0])
    
        trainC, valC = dataset_handler.prepare_data_CatBoost(x_train[:,:,2:], y_train, x_val[:,:,2:], y_val)
        trainL, valL = dataset_handler.prepare_data_LSTM(x_train[:,:,2:], y_train, x_val[:,:,2:], y_val)

        catBoost_train = cat_boost.model.predict(trainC[0])
        catBoost_train[catBoost_train < 0] = 0
        catBoost_val = cat_boost.model.predict(valC[0])
        catBoost_val[catBoost_val < 0] = 0

        svm_train = svm.model.predict(trainC[0])
        svm_train[svm_train < 0] = 0
        svm_val = svm.model.predict(valC[0])
        svm_val[svm_val < 0] = 0

        lstm_train = lstm.model.predict(trainL[0])
        lstm_train[lstm_train < 0] = 0
        lstm_val = lstm.model.predict(valL[0])
        lstm_val[lstm_val < 0] = 0

        train_pol0, train_pol1, train_pol2, val_pol0, val_pol1, val_pol2 = interpolation(trainL, valL)

        x_trainE = np.concatenate((catBoost_train, svm_train, lstm_train, train_pol0, train_pol1, train_pol2), axis=-1)
        y_trainE = trainingL[1]
        x_valE = np.concatenate((catBoost_val, svm_val, lstm_val, val_pol0, val_pol1, val_pol2), axis=-1)
        y_valE = validationL[1]

        preds_tra = ens.model.predict(x_trainE)
        preds_tra[preds_tra < 0] = 0
        preds_val = ens.model.predict(x_valE)
        preds_val[preds_val < 0] = 0

        datelist = pd.date_range('01-01-2010', end='01-01-2020', freq='YS')
        dates = datelist.strftime("%Y").tolist()
        # Total timesteps (nb of months inside this time window) = 228
        month_datelist = pd.date_range('01-01-2010', end='31-12-2019', freq='MS')
        ts = np.arange(0, len(month_datelist), 1)

        res = []
        res.append('Department,NRMSE 0-19 Training,NRMSE All Training,NRMSE 0-19 Validation,NRMSE All Validation')

        for i in range(nb_deps):
            fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(18,9))

            idxt = preds_tra.shape[0]//nb_deps
            idxv = preds_val.shape[0]//nb_deps        

            t1 = ts[12:12+idxt]
            t2 = ts[(12+idxt):(12+idxt+idxv)]

            # Reverting Data Norm ---------
            pt = scaler.inverse_transform(preds_tra[idxt*i:idxt*(i+1), :])
            pv = scaler.inverse_transform(preds_val[idxv*i:idxv*(i+1), :])

            gtt = scaler.inverse_transform(trainL[1][idxt*i:idxt*(i+1), :])
            gtv = scaler.inverse_transform(valL[1][idxv*i:idxv*(i+1), :])
            # ------------------------------

            baseAll_df = baselineAll[baselineAll.state_index == i][['pred.mean','pred.uci','pred.lci']]
            base019_df = baseline019[baseline019.state_index == i][['pred.mean','pred.uci','pred.lci']]

            axes[0].plot(t1, gtt[:,0], '-', color='orange', label = 'Observed Cases')
            axes[0].plot(t2, gtv[:,0], '-', color='orange')
            axes[0].plot(t1, pt[:,0], '--', color='red', label = 'Ensemble prediction on Training data')
            axes[0].plot(t2, pv[:,0], '-', color='red', label = 'Ensemble prediction on Validation data')

            # Baseline model
            axes[0].plot(t1, baseAll_df['pred.mean'].iloc[t1], color='forestgreen', label = 'Baseline model prediction')
            axes[0].fill_between(x=t1, y1=baseAll_df['pred.lci'].iloc[t1], y2=baseAll_df['pred.uci'].iloc[t1], facecolor='forestgreen', alpha=.1, label='Baseline model Confidence Interval')
            axes[0].plot(t2, baseAll_df['pred.mean'].iloc[t2], color='forestgreen')
            axes[0].fill_between(x=t2, y1=baseAll_df['pred.lci'].iloc[t2], y2=baseAll_df['pred.uci'].iloc[t2], facecolor='forestgreen', alpha=.1)

            axes[1].plot(t1, gtt[:,1], '-', color='orange', label = 'Observed Cases')
            axes[1].plot(t2, gtv[:,1], '-', color='orange')
            axes[1].plot(t1, pt[:,1], '--', color='red', label = 'Ensemble prediction on Training data')
            axes[1].plot(t2, pv[:,1], '-', color='red', label = 'Ensemble prediction on Validation data')

            # Baseline model
            axes[1].plot(t1, base019_df['pred.mean'].iloc[t1], color='forestgreen', label = 'Baseline model prediction')
            axes[1].fill_between(x=t1, y1=base019_df['pred.lci'].iloc[t1], y2=base019_df['pred.uci'].iloc[t1], facecolor='forestgreen', alpha=.1, label='Baseline model Confidence Interval')
            axes[1].plot(t2, base019_df['pred.mean'].iloc[t2], color='forestgreen')
            axes[1].fill_between(x=t2, y1=base019_df['pred.lci'].iloc[t2], y2=base019_df['pred.uci'].iloc[t2], facecolor='forestgreen', alpha=.1)

            # Configure visualization
            axes[0].set_title(DEP_NAMES[i], fontsize = 18, loc='left')
            axes[0].set(ylabel='DIR Total Population')
            axes[1].set(xlabel='Year', ylabel='DIR 0-19')
            axes[1].legend(ncol=3, loc='lower center', bbox_to_anchor=(0.5,-0.5), shadow=True)
            axes[0].axvline(t2[0], color='black', linewidth=2, linestyle='--', alpha=0.5)
            axes[1].axvline(t2[0], color='black', linewidth=2, linestyle='--', alpha=0.5)

            for ax in axes:
                ax.set_xticks(np.arange(0, len(ts)+1, 12))
                ax.grid(True, linewidth=0.5)
                ax.set_xlim(ts[0],ts[-1]+5)
            axes[0].set_xticklabels([])
            axes[1].set_xticklabels(dates)

            fig.tight_layout()
            plt.show()
            plt.close()

            #print('--------------------- TRAINING scores ---------------------')
            #rmse1, r2 = compute_scores(gtt[:,0], pt[:,0])
            #print('N-RMSE (group total): ', rmse1)
            #print('R2   (group total): ', r2)

            #rmse2, r2 = compute_scores(gtt[:,1], pt[:,1])
            #print('N-RMSE (group 0-19): ', rmse2)
            #print('R2   (group 0-19): ', r2)

            print('-------------------- VALIDATION scores Proposed ---------------------')
            rmse3, r2 = compute_scores(gtv[:,0], pv[:,0])
            print('N-RMSE (group total): ', rmse3)
            print('R2   (group total): ', r2)

            rmse4, r2 = compute_scores(gtv[:,1], pv[:,1])
            print('N-RMSE (group 0-19): ', rmse4)
            print('R2   (group 0-19): ', r2)

            print('-------------------- VALIDATION scores Baseline ---------------------')
            rmse3, r2 = compute_scores(gtv[:,0], baseAll_df['pred.mean'].iloc[t2])
            print('N-RMSE (group total): ', rmse3)
            print('R2   (group total): ', r2)

            rmse4, r2 = compute_scores(gtv[:,1], base019_df['pred.mean'].iloc[t2])
            print('N-RMSE (group 0-19): ', rmse4)
            print('R2   (group 0-19): ', r2)

            #res.append('{},{},{},{},{}'.format(DEP_NAMES[i], rmse2, rmse1, rmse4, rmse3))

            print('\n##########################################################################################################')

        # Save results
        #today = datetime.now().strftime("%d-%m-%Y-%H:%M:%S")
        # with open(join(config['metrics'], "Peru", 'rf_ensemble-'+today+'.csv'), 'w') as f:
        #     print('Writing performance metrics to file...')
        #     for item in res:
        #         f.write("%s\n" % item)

Compute confidence intervals for the ensemble model.

In [ ]:
# Inner Brazilian models
cat_boost.load(glob(join(config['output'], "Brazil", "catboost*"))[0])
svm.load(glob(join(config['output'], "Brazil", "svm*"))[0])
lstm.load(glob(join(config['output'], "Brazil", "lstm*"))[0])

# Finetuned Ensemble in Peru
ens.load(glob(join(config['output'], "Peru", "rf*"))[0])

In [ ]:
trainC, valC = dataset_handler.prepare_data_CatBoost(x_train[:,:,2:], y_train, x_val[:,:,2:], y_val)
trainL, valL = dataset_handler.prepare_data_LSTM(x_train[:,:,2:], y_train, x_val[:,:,2:], y_val)

catBoost_train = cat_boost.model.predict(trainC[0])
catBoost_train[catBoost_train < 0] = 0
catBoost_val = cat_boost.model.predict(valC[0])
catBoost_val[catBoost_val < 0] = 0

svm_train = svm.model.predict(trainC[0])
svm_train[svm_train < 0] = 0
svm_val = svm.model.predict(valC[0])
svm_val[svm_val < 0] = 0

lstm_train = lstm.model.predict(trainL[0])
lstm_train[lstm_train < 0] = 0
lstm_val = lstm.model.predict(valL[0])
lstm_val[lstm_val < 0] = 0

train_pol0, train_pol1, train_pol2, val_pol0, val_pol1, val_pol2 = interpolation(trainL, valL)

x_trainE = np.concatenate((catBoost_train, svm_train, lstm_train, train_pol0, train_pol1, train_pol2), axis = -1)
y_trainE = trainingL[1]
x_valE = np.concatenate((catBoost_val, svm_val, lstm_val, val_pol0, val_pol1, val_pol2), axis = -1)
y_valE = validationL[1]

print(x_trainE.shape, y_trainE.shape, x_valE.shape, y_valE.shape)

In [39]:
def pred_ints(model, X, percentile=95):
    err_down = []
    err_up = []
    for x in range(len(X)):
        preds = []
        for pred in model.estimators_:
            preds.append(pred.predict(X[x].reshape(1, -1))[0])

        preds = np.array(preds)
        err_down.append(np.percentile(preds, (100 - percentile) / 2., axis=0))
        err_up.append(np.percentile(preds, 100 - (100 - percentile) / 2., axis=0))
    return np.array(err_down), np.array(err_up)

train_err_down, train_err_up = pred_ints(ens.model, x_trainE, percentile=95)
val_err_down, val_err_up = pred_ints(ens.model, x_valE, percentile=95)

In [ ]:
preds_tra = ens.model.predict(x_trainE)
preds_tra[preds_tra < 0] = 0
preds_val = ens.model.predict(x_valE)
preds_val[preds_val < 0] = 0

datelist = pd.date_range('01-01-2010', end='01-01-2020', freq='YS')
dates = datelist.strftime("%Y").tolist()
# Total timesteps (nb of months inside this time window) = 228
month_datelist = pd.date_range('01-01-2010', end='31-12-2019', freq='MS')
ts = np.arange(0, len(month_datelist), 1)

for i in range(nb_deps):
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(18,9))

    idxt = preds_tra.shape[0]//nb_deps
    idxv = preds_val.shape[0]//nb_deps

    t1 = ts[12:12+idxt]
    t2 = ts[(12+idxt):(12+idxt+idxv)]

    # Reverting Data Norm ---------
    pt = scaler.inverse_transform(preds_tra[idxt*i:idxt*(i+1), :])
    pv = scaler.inverse_transform(preds_val[idxv*i:idxv*(i+1), :])

    gtt = scaler.inverse_transform(trainL[1][idxt*i:idxt*(i+1), :])
    gtv = scaler.inverse_transform(valL[1][idxv*i:idxv*(i+1), :])

    lstm_pt = scaler.inverse_transform(lstm_train[idxt*i:idxt*(i+1), :])
    lstm_pv = scaler.inverse_transform(lstm_val[idxv*i:idxv*(i+1), :])

    svm_pt = scaler.inverse_transform(svm_train[idxt*i:idxt*(i+1), :])
    svm_pv = scaler.inverse_transform(svm_val[idxv*i:idxv*(i+1), :])

    cb_pt = scaler.inverse_transform(catBoost_train[idxt*i:idxt*(i+1), :])
    cb_pv = scaler.inverse_transform(catBoost_val[idxv*i:idxv*(i+1), :])

    t_lci = scaler.inverse_transform(train_err_down[idxt*i:idxt*(i+1), :])
    t_uci = scaler.inverse_transform(train_err_up[idxt*i:idxt*(i+1), :])

    v_lci = scaler.inverse_transform(val_err_down[idxv*i:idxv*(i+1), :])
    v_uci = scaler.inverse_transform(val_err_up[idxv*i:idxv*(i+1), :])
    # ------------------------------
    
    baseAll_df = baselineAll[baselineAll.state_index == i][['pred.mean','pred.uci','pred.lci']]
    base019_df = baseline019[baseline019.state_index == i][['pred.mean','pred.uci','pred.lci']]

    ########################
    ### Total population ###
    ########################
    axes[0].plot(t1, gtt[:,0], '-', color='orange', label = 'Observed Cases')
    axes[0].plot(t2, gtv[:,0], '-', color='orange')
    axes[0].plot(t1, pt[:,0], '--', color='red', label = 'Ensemble prediction on Training data')
    axes[0].fill_between(t1, y1=(pt[:,0]-t_lci[:,0]), y2=(pt[:,0]+t_uci[:,0]), facecolor='red', color='red', linewidth=0.5, alpha=.1, label='Ensemble CI')
    axes[0].plot(t2, pv[:,0], '-', color='red', label = 'Ensemble prediction on Validation data')
    axes[0].fill_between(t2, y1=(pv[:,0]-v_lci[:,0]), y2=(pv[:,0]+v_uci[:,0]), facecolor='red', color='red', linewidth=0.5, alpha=.1)

    # Baseline model
    axes[0].plot(t1, baseAll_df['pred.mean'].iloc[t1], color='forestgreen', label = 'Baseline model prediction')
    axes[0].fill_between(x=t1, y1=baseAll_df['pred.lci'].iloc[t1], y2=baseAll_df['pred.uci'].iloc[t1], facecolor='forestgreen', color='forestgreen', linewidth=0.5, alpha=.1, label='Baseline model CI')
    axes[0].plot(t2, baseAll_df['pred.mean'].iloc[t2], color='forestgreen')
    axes[0].fill_between(x=t2, y1=baseAll_df['pred.lci'].iloc[t2], y2=baseAll_df['pred.uci'].iloc[t2], facecolor='forestgreen', color='forestgreen', linewidth=0.5, alpha=.1)

    #####################
    ### Children 0-19 ###
    #####################
    axes[1].plot(t1, gtt[:,1], '-', color='orange', label = 'Observed Cases')
    axes[1].plot(t2, gtv[:,1], '-', color='orange')
    axes[1].plot(t1, pt[:,1], '--', color='red', label = 'Ensemble prediction on Training data')
    axes[1].fill_between(t1, y1=(pt[:,1]-t_lci[:,1]), y2=(pt[:,1]+t_uci[:,1]), facecolor='red', color='red', linewidth=0.5, alpha=.1, label='Ensemble CI')
    axes[1].plot(t2, pv[:,1], '-', color='red', label = 'Ensemble prediction on Validation data')
    axes[1].fill_between(t2, y1=(pv[:,1]-v_lci[:,1]), y2=(pv[:,1]+v_uci[:,1]), facecolor='red', color='red', linewidth=0.5, alpha=.1)

    # Baseline model
    axes[1].plot(t1, base019_df['pred.mean'].iloc[t1], color='forestgreen', label = 'Baseline model prediction')
    axes[1].fill_between(x=t1, y1=base019_df['pred.lci'].iloc[t1], y2=base019_df['pred.uci'].iloc[t1], facecolor='forestgreen', color='forestgreen', linewidth=0.5, alpha=.1, label='Baseline model CI')
    axes[1].plot(t2, base019_df['pred.mean'].iloc[t2], color='forestgreen')
    axes[1].fill_between(x=t2, y1=base019_df['pred.lci'].iloc[t2], y2=base019_df['pred.uci'].iloc[t2], facecolor='forestgreen', color='forestgreen', linewidth=0.5, alpha=.1)

    # Configure visualization
    axes[0].set_title(DEP_NAMES[i], fontsize = 18, loc='left', fontweight="bold")
    axes[0].set(ylabel='DIR Total Population')
    axes[1].set(xlabel='Year', ylabel='DIR 0-19')
    # if i in prob_deps:
    #     axes[1].legend(ncol=5, loc='lower center', bbox_to_anchor=(0.5,-0.5), shadow=True, fontsize=12.5)
    # else:
    axes[1].legend(ncol=3, loc='lower center', bbox_to_anchor=(0.5,-0.5), shadow=True)
    axes[0].axvline(t2[0], color='black', linewidth=2, linestyle='--', alpha=0.5)
    axes[1].axvline(t2[0], color='black', linewidth=2, linestyle='--', alpha=0.5)

    for ax in axes:
        #ax.set_xticks(np.arange(0, idxt+idxv, 11))
        ax.set_xticks(np.arange(0, len(ts)+1, 12))
        ax.grid(True, linewidth=0.5)
        ax.set_xlim(ts[0],ts[-1]+5)
    axes[0].set_xticklabels([])
    axes[1].set_xticklabels(dates)

    fig.tight_layout()
    plt.show()
    plt.close()
    print('\n##########################################################################################################')

**Get time series plots with validation period zoomed.**

In [ ]:
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
y_rescaled = False

preds_tra = ens.model.predict(x_trainE)
preds_tra[preds_tra < 0] = 0
preds_val = ens.model.predict(x_valE)
preds_val[preds_val < 0] = 0

datelist = pd.date_range('01-01-2010', end='01-01-2020', freq='YS')
dates = datelist.strftime("%Y").tolist()

# Total timesteps (nb of months inside this time window) = 228
month_datelist = pd.date_range('01-01-2010', end='31-12-2019', freq='MS')
ts = np.arange(0, len(month_datelist), 1)
val_dates = pd.date_range('01-01-2017', end='31-12-2019', freq = 'M').strftime("%m-%Y").tolist()


for i in range(nb_deps):
    fig, (axes1, axes2) = plt.subplots(nrows=2, ncols=2, figsize=(22,9), gridspec_kw={'width_ratios': [6, 3], 'wspace':0.01, 'hspace':0.06})

    idxt = preds_tra.shape[0]//nb_deps
    idxv = preds_val.shape[0]//nb_deps        

    t1 = ts[12:12+idxt]
    t2 = ts[(12+idxt):(12+idxt+idxv)]

    # Reverting Data Norm ---------
    pt = scaler.inverse_transform(preds_tra[idxt*i:idxt*(i+1), :])
    pv = scaler.inverse_transform(preds_val[idxv*i:idxv*(i+1), :])

    gtt = scaler.inverse_transform(trainL[1][idxt*i:idxt*(i+1), :])
    gtv = scaler.inverse_transform(valL[1][idxv*i:idxv*(i+1), :])

    t_lci = scaler.inverse_transform(train_err_down[idxt*i:idxt*(i+1), :])
    t_uci = scaler.inverse_transform(train_err_up[idxt*i:idxt*(i+1), :])

    v_lci = scaler.inverse_transform(val_err_down[idxv*i:idxv*(i+1), :])
    v_uci = scaler.inverse_transform(val_err_up[idxv*i:idxv*(i+1), :])
    # ------------------------------
    
    baseAll_df = baselineAll[baselineAll.state_index == i][['pred.mean','pred.uci','pred.lci']]
    base019_df = baseline019[baseline019.state_index == i][['pred.mean','pred.uci','pred.lci']]

    #________________________________________________________________________
    # TOTAL POPULATION
    axes1[0].plot(t1, gtt[:,0], '-', color='orange', label = 'Observed Cases')
    axes1[0].plot(t2, gtv[:,0], '-', color='orange')
    axes1[0].plot(t1, pt[:,0], '-', color='red', label = 'Ensemble prediction')
    axes1[0].fill_between(t1, y1=(pt[:,0]-t_lci[:,0]), y2=(pt[:,0]+t_uci[:,0]), facecolor='red', linewidth=1, alpha=.1, label='Ensemble CI')
    axes1[0].plot(t2, pv[:,0], '-', color='red', label = 'Ensemble prediction')
    axes1[0].fill_between(t2, y1=(pv[:,0]-v_lci[:,0]), y2=(pv[:,0]+v_uci[:,0]), facecolor='red', linewidth=1, alpha=.1)

    # Baseline model
    axes1[0].plot(t1, baseAll_df['pred.mean'].iloc[t1], color='forestgreen', label = 'Baseline model prediction')
    axes1[0].fill_between(x=t1, y1=baseAll_df['pred.lci'].iloc[t1], y2=baseAll_df['pred.uci'].iloc[t1], facecolor='forestgreen', alpha=.1, label='Baseline model CI')
    axes1[0].plot(t2, baseAll_df['pred.mean'].iloc[t2], color='forestgreen')
    axes1[0].fill_between(x=t2, y1=baseAll_df['pred.lci'].iloc[t2], y2=baseAll_df['pred.uci'].iloc[t2], facecolor='forestgreen', alpha=.1)

    # Validation Zoom
    axes1[1].plot(t2, gtv[:,0], '-', color='orange')
    axes1[1].plot(t2, pv[:,0], '-', color='red')
    axes1[1].fill_between(t2, y1=(pv[:,0]-v_lci[:,0]), y2=(pv[:,0]+v_uci[:,0]), facecolor='red', alpha=.1)
    axes1[1].plot(t2, baseAll_df['pred.mean'].iloc[t2], color='forestgreen')
    axes1[1].fill_between(x=t2, y1=baseAll_df['pred.lci'].iloc[t2], y2=baseAll_df['pred.uci'].iloc[t2], facecolor='forestgreen', alpha=.1)

    #________________________________________________________________________
    # AGE 0-19
    axes2[0].plot(t1, gtt[:,1], '-', color='orange', label = 'Observed Cases')
    axes2[0].plot(t2, gtv[:,1], '-', color='orange')
    axes2[0].plot(t1, pt[:,1], '-', color='red', label = 'Ensemble prediction')
    axes2[0].fill_between(t1, y1=(pt[:,1]-t_lci[:,1]), y2=(pt[:,1]+t_uci[:,1]), facecolor='red', alpha=.1, label='Ensemble CI')
    axes2[0].plot(t2, pv[:,1], '-', color='red', label = 'Ensemble prediction')
    axes2[0].fill_between(t2, y1=(pv[:,1]-v_lci[:,1]), y2=(pv[:,1]+v_uci[:,1]), facecolor='red', alpha=.1)

    # Baseline model
    axes2[0].plot(t1, base019_df['pred.mean'].iloc[t1], color='forestgreen', label = 'Baseline model prediction')
    axes2[0].fill_between(x=t1, y1=base019_df['pred.lci'].iloc[t1], y2=base019_df['pred.uci'].iloc[t1], facecolor='forestgreen', alpha=.1, label='Baseline model CI')
    axes2[0].plot(t2, base019_df['pred.mean'].iloc[t2], color='forestgreen')
    axes2[0].fill_between(x=t2, y1=base019_df['pred.lci'].iloc[t2], y2=base019_df['pred.uci'].iloc[t2], facecolor='forestgreen', alpha=.1)

    # Validation Zoom
    axes2[1].plot(t2, gtv[:,1], '-', color='orange')
    axes2[1].plot(t2, pv[:,1], '-', color='red')
    axes2[1].fill_between(t2, y1=(pv[:,1]-v_lci[:,1]), y2=(pv[:,1]+v_uci[:,1]), facecolor='red', alpha=.1)
    axes2[1].plot(t2, base019_df['pred.mean'].iloc[t2], color='forestgreen')
    axes2[1].fill_between(x=t2, y1=base019_df['pred.lci'].iloc[t2], y2=base019_df['pred.uci'].iloc[t2], facecolor='forestgreen', alpha=.1)

    # Configure visualization
    axes1[0].set_title(DEP_NAMES[i], fontsize = 18, loc='left', fontweight="bold")
    axes1[1].set_title('Validation period', fontsize = 15)
    axes1[0].set(ylabel='DIR Total Population')
    axes2[0].set(xlabel='Year', ylabel='DIR 0-19')
    axes2[1].set(xlabel='Month-Year')
    axes1[0].axvline(t2[0], color='black', linewidth=2, linestyle='--', alpha=0.5)
    axes2[0].axvline(t2[0], color='black', linewidth=2, linestyle='--', alpha=0.5)

    legend_elements = [Line2D([0], [0], color='orange', lw=2),
                       (Line2D([0], [0], color='red', lw=2), Patch(facecolor='red', alpha=.1, lw=2)),
                       (Line2D([0], [0], color='forestgreen', lw=2), Patch(facecolor='forestgreen', alpha=.1, lw=2))                             
                    ]
    axes2[0].legend(handles=legend_elements, labels=['Observed Cases', 'Ensemble Prediction + CI', 'Baseline Prediction + CI'], 
                    ncol=3, loc='lower center', bbox_to_anchor=(0.75,-0.55), shadow=True)


    for ax in [axes1[0], axes2[0]]:
        #ax.set_yscale('log')
        ax.set_xticks(np.arange(0, len(ts)+1, 12))
        ax.grid(True, linewidth=0.5)
        ax.set_xlim(ts[0],ts[-1]+5)

    for ax in [axes1[1], axes2[1]]:
        #ax.set_yscale('log')
        ax.set_xticks(np.arange(t2[0],t2[0]+len(t2),3))
        ax.yaxis.tick_right()
        ax.grid(True, linewidth=0.5)
        ax.set_xlim(t2[0]-1,)

    if y_rescaled:
        ymax = max(np.max(gtt[:,0]), np.max(gtv[:,0]), np.max(pt[:,0]), np.max(pv[:,0]))
        ymax_019 = max(np.max(gtt[:,1]), np.max(gtv[:,1]), np.max(pt[:,1]), np.max(pv[:,1]))
        ymin = min(np.min(gtt[:,0]), np.min(gtv[:,0]), np.min(pt[:,0]), np.min(pv[:,0]))
        ymin_019 = min(np.min(gtt[:,1]), np.min(gtv[:,1]), np.min(pt[:,1]), np.min(pv[:,1]))

        val_ymax = max(np.max(gtv[:,0]), np.max(pv[:,0]))
        val_ymax_019 = max(np.max(gtv[:,1]), np.max(pv[:,1]))
        val_ymin = min(np.min(gtv[:,0]), np.min(pv[:,0]))
        val_ymin_019 = min(np.min(gtv[:,1]), np.min(pv[:,1]))

        # 5% bottom margin and 10% top margin
        minY = ymin-0.05*(ymax-ymin)
        maxY = ymax+0.1*(ymax-ymin)
        minY_019 = ymin_019-0.05*(ymax_019-ymin_019)
        maxY_019 = ymax_019+0.1*(ymax_019-ymin_019)
        val_minY = val_ymin-0.05*(val_ymax-val_ymin)
        val_maxY = val_ymax+0.1*(val_ymax-val_ymin)
        val_minY_019 = val_ymin_019-0.05*(val_ymax_019-val_ymin_019)
        val_maxY_019 = val_ymax_019+0.1*(val_ymax_019-val_ymin_019)

        axes1[0].set_ylim((minY, maxY))
        axes2[0].set_ylim((minY_019, maxY_019))
        axes1[1].set_ylim((val_minY, val_maxY))
        axes2[1].set_ylim((val_minY_019, val_maxY_019))

    axes1[0].set_xticklabels([])
    axes2[0].set_xticklabels(dates)
    axes1[1].set_xticklabels([])
    axes2[1].set_xticklabels(val_dates[::3], rotation=45)  

    plt.show()
    plt.close()